In [1]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=23631ce0748d6f0e604139f23e08da0c5a2caa4014c641233850f6ea3302337e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import findspark
findspark.init()
from pyspark.sql import SQLContext, SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("Combine CASTNET Files")
         .config("spark.ui.port", "4050")
         .getOrCreate())

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [4]:
import os
import shutil
import sys
import time
import numpy as np
import pandas as pd
from google.colab import drive

drive_root = "/content/drive"
drive.mount(drive_root)

file_directory = os.path.join(drive_root, "MyDrive", "AML Group 24", "Files")
os.listdir(file_directory)

FILE_OUTPUT = "output"
if not os.path.exists(FILE_OUTPUT):
    os.makedirs(FILE_OUTPUT)

Mounted at /content/drive


In [5]:
def save_df_to_parquet(df, output_folder):
  df.coalesce(1).write.format("parquet").mode("append").save(os.path.join(FILE_OUTPUT, output_folder))

  parquet_file = [i for i in os.listdir(os.path.join(FILE_OUTPUT, output_folder)) if i.endswith("parquet")]
  assert len(parquet_file) == 1, "Did you run this cell multiple times?"
  parquet_file = parquet_file[0]

  shutil.move(os.path.join(FILE_OUTPUT, output_folder, parquet_file), os.path.join(FILE_OUTPUT, f"{output_folder}.snappy.parquet"))
  shutil.rmtree(os.path.join(FILE_OUTPUT, output_folder))

  print(os.stat(os.path.join(FILE_OUTPUT, f"{output_folder}.snappy.parquet")).st_size/(1024**2))

# Site

In [12]:
site_options = {
    "header": True,
}

schema = StructType()
schema.add("SITE_ID", StringType(), False)
schema.add("SITE_NUM", DecimalType(38,0))
schema.add("SITE_NAME", StringType())
schema.add("ACTIVE", StringType())
schema.add("INACTIVE", StringType())
schema.add("AGENCY", StringType())
schema.add("STATE", StringType())
schema.add("COUNTY", StringType())
schema.add("TIME_ZONE", StringType())
schema.add("LATITUDE", DecimalType(14,6))
schema.add("LONGITUDE", DecimalType(14,6))
schema.add("ELEVATION", DecimalType(20,5))
schema.add("MAPID", StringType())
schema.add("LAND_USE", StringType())
schema.add("TERRAIN", StringType())
schema.add("MLM", StringType())
schema.add("NADP_ID", StringType())
schema.add("NADP_DISTANCE", DecimalType(12,4))
schema.add("UPDATE_DATE", StringType())

site_file = os.path.join(file_directory, "site.csv")
df_site = spark.read.format("csv").schema(schema).options(**site_options).load(site_file)

df_site.printSchema()


site_columns_to_keep = ["SITE_ID", "LATITUDE", "LONGITUDE", "ELEVATION", "LAND_USE", "TERRAIN"]

df_site = df_site.select([c for c in df_site.columns if c in site_columns_to_keep])

df_site.printSchema()

root
 |-- SITE_ID: string (nullable = true)
 |-- SITE_NUM: decimal(38,0) (nullable = true)
 |-- SITE_NAME: string (nullable = true)
 |-- ACTIVE: string (nullable = true)
 |-- INACTIVE: string (nullable = true)
 |-- AGENCY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- TIME_ZONE: string (nullable = true)
 |-- LATITUDE: decimal(14,6) (nullable = true)
 |-- LONGITUDE: decimal(14,6) (nullable = true)
 |-- ELEVATION: decimal(20,5) (nullable = true)
 |-- MAPID: string (nullable = true)
 |-- LAND_USE: string (nullable = true)
 |-- TERRAIN: string (nullable = true)
 |-- MLM: string (nullable = true)
 |-- NADP_ID: string (nullable = true)
 |-- NADP_DISTANCE: decimal(12,4) (nullable = true)
 |-- UPDATE_DATE: string (nullable = true)

root
 |-- SITE_ID: string (nullable = true)
 |-- LATITUDE: decimal(14,6) (nullable = true)
 |-- LONGITUDE: decimal(14,6) (nullable = true)
 |-- ELEVATION: decimal(20,5) (nullable = true)
 |-- LAND_USE: strin

In [11]:
display(df_site)

SITE_ID,LATITUDE,LONGITUDE,ELEVATION,LAND_USE,TERRAIN
ABT147,41.840460,-72.010368,202.00000,Urban/Agric,Rolling
ACA416,44.377086,-68.260800,158.00000,Forest,Rolling
ALB801,53.682400,-112.868000,711.00000,Forest,Flat
ALC188,30.701577,-94.674011,105.00000,Prairie,Rolling
ALH157,38.869000,-89.622800,164.00000,Agric,Flat
ALH257,38.869001,-89.622815,164.00000,Agric,Flat
ALH557,38.869001,-89.622815,164.00000,Agric,Flat
ANA115,42.416636,-83.902180,266.00000,Forest,Flat
ANL146,41.700000,-87.990000,229.00000,Urban/Agric,Rolling
ARE128,39.923241,-77.307863,266.00000,Agric,Rolling


In [32]:
output_folder = "site"
save_df_to_parquet(df_site, output_folder)

0.005766868591308594


# Hourly Gas

In [13]:
gas_file = os.path.join(file_directory, "hourly_gas_combined.snappy.parquet")
df_gas = spark.read.format("parquet").load(gas_file)

# print(df_gas.count())

df_gas = df_gas.where(~F.col("PARAMETER").isin(["O3_23M", "TEMP_23M", "O3_10M", "O3_14M", "O3_17M", "O3_20M", "O3_28M", "O3_2M", "O3_6M", "TEMP_2M"]))

df_gas = df_gas.where(F.col("QA_CODE").isin(["1", "3"]))
df_gas = df_gas.where(F.col("VALUE_F").isNull() | F.col("VALUE_F").isin("<", "A", "S", "U", "Z"))

# print(df_gas.count())

df_gas = df_gas.select(["SITE_ID", "DATE_TIME", "PARAMETER", "VALUE"]) # needs to be pivoted
print(f"Original Count: {df_gas.count()}")

df_just_2_pk = df_gas.select("SITE_ID", "DATE_TIME").distinct()
print(f"Number of rows in pivot: {df_just_2_pk.count()}")

Original Count: 2475297
Number of rows in pivot: 624865


In [14]:
df_gas_pivot = df_gas.groupBy("SITE_ID", "DATE_TIME").pivot("PARAMETER").sum("VALUE")
print(f"Pivot Count: {df_gas_pivot.count()}")

Pivot Count: 624865


In [15]:
output_folder = "gas_pivot"
save_df_to_parquet(df_gas_pivot, output_folder)

10.484942436218262


In [16]:
del df_just_2_pk
del df_gas_pivot

In [17]:
spark.read.format("parquet").load("output/gas_pivot.snappy.parquet").count()

624865

# Met

* We will just filter our label OZONE by the OZONE_F flag for valid date
* For deeper analysis, we will see which features we can completely drop before we apply the corresponding _F filter

In [18]:
met_file = os.path.join(file_directory, "metdata_combined.snappy.parquet")
df_met = spark.read.format("parquet").load(met_file)

# print(df_met.count())

df_met = df_met.filter(F.col("OZONE_F").isNull() | F.col("OZONE_F").isin(["<", "E", "K", "S", "^", "_"]))
# print(df_met.count())

df_met = df_met.where(F.col("QA_CODE") != "X")

print(df_met.count())

# Other _F
#~ (['&', '<', 'A', 'E', 'K', 'Q', 'S', 'W', 'Y', '^', '_']

7017729


In [19]:
output_folder = "met"
save_df_to_parquet(df_met, output_folder)

92.95935249328613


In [20]:
spark.read.format("parquet").load("output/met.snappy.parquet").count()

7017729

In [21]:
del df_met

# Join met with gas

In [22]:
df_gas_pivot = spark.read.format("parquet").load(os.path.join("output", "gas_pivot.snappy.parquet"))
df_met = spark.read.format("parquet").load(os.path.join("output", "met.snappy.parquet"))

df_met_gas = df_met.join(df_gas_pivot, how="inner", on=["SITE_ID", "DATE_TIME"])

df_met_gas.count()

574851

In [23]:
len(df_met_gas.columns)

46

In [24]:
output_folder = "met_gas"
save_df_to_parquet(df_met_gas, output_folder)

17.029431343078613


In [25]:
del df_met_gas

In [26]:
spark.read.format("parquet").load("output/met_gas.snappy.parquet").count()

574851

# Join met with gas with site

In [27]:
met_gas_file = os.path.join("output", "met_gas.snappy.parquet")
df_met_gas = spark.read.format("parquet").load(met_gas_file)

df_met_gas_site = df_met_gas.join(df_site, how="inner", on=["SITE_ID"])
df_met_gas_site.count()

574851

In [28]:
df_met_gas_site.printSchema()

root
 |-- SITE_ID: string (nullable = true)
 |-- DATE_TIME: timestamp (nullable = true)
 |-- TEMPERATURE: decimal(16,4) (nullable = true)
 |-- TEMPERATURE_F: string (nullable = true)
 |-- TEMPERATURE_DELTA: decimal(16,4) (nullable = true)
 |-- TEMPERATURE_DELTA_F: string (nullable = true)
 |-- RELATIVE_HUMIDITY: decimal(16,4) (nullable = true)
 |-- RELATIVE_HUMIDITY_F: string (nullable = true)
 |-- SOLAR_RADIATION: decimal(16,4) (nullable = true)
 |-- SOLAR_RADIATION_F: string (nullable = true)
 |-- OZONE: decimal(16,4) (nullable = true)
 |-- OZONE_F: string (nullable = true)
 |-- PRECIPITATION: decimal(16,4) (nullable = true)
 |-- PRECIPITATION_F: string (nullable = true)
 |-- WINDSPEED: decimal(16,4) (nullable = true)
 |-- WINDSPEED_F: string (nullable = true)
 |-- WIND_DIRECTION: decimal(16,4) (nullable = true)
 |-- WIND_DIRECTION_F: string (nullable = true)
 |-- SIGMA_THETA: decimal(16,4) (nullable = true)
 |-- SIGMA_THETA_F: string (nullable = true)
 |-- FLOW_RATE: decimal(16,4) (

In [29]:
output_folder = "met_gas_site"
save_df_to_parquet(df_met_gas_site, output_folder)

17.274246215820312


In [30]:
spark.read.format("parquet").load("output/met_gas_site.snappy.parquet").count()

574851

In [31]:
result = spark.read.format("parquet").load("output/met_gas_site.snappy.parquet")
display(result)

SITE_ID,DATE_TIME,TEMPERATURE,TEMPERATURE_F,TEMPERATURE_DELTA,TEMPERATURE_DELTA_F,RELATIVE_HUMIDITY,RELATIVE_HUMIDITY_F,SOLAR_RADIATION,SOLAR_RADIATION_F,OZONE,OZONE_F,PRECIPITATION,PRECIPITATION_F,WINDSPEED,WINDSPEED_F,WIND_DIRECTION,WIND_DIRECTION_F,SIGMA_THETA,SIGMA_THETA_F,FLOW_RATE,FLOW_RATE_F,WINDSPEED_SCALAR,WINDSPEED_SCALAR_F,WETNESS,WETNESS_F,SHELTER_TEMPERATURE,SHELTER_TEMPERATURE_F,QA_CODE,UPDATE_DATE,Filename,CO,FLOW_23M,FLOW_2M,HNO3,NH3,NO,NO2_TRUE,NOX,NOXDIF,NOX_TRUE,NOY,NOYDIF,NOY_MINUS,SO2_GA,TNX,LATITUDE,LONGITUDE,ELEVATION,LAND_USE,TERRAIN
BEL116,2013-01-01 22:00:00,3.8590,NULL,0.0440,NULL,67.7900,NULL,0.4750,NULL,23.3200,NULL,0.0000,NULL,1.0080,NULL,318.5000,NULL,36.9700,NULL,1.5150,NULL,1.1930,NULL,0.0000,NULL,27.0500,NULL,3,2013-01-01 22:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.0290,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0.9990,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-02 03:00:00,1.5730,NULL,0.0570,NULL,63.0100,NULL,0.4710,NULL,30.5300,NULL,0.0000,NULL,0.7250,NULL,5.0380,NULL,38.4400,NULL,1.5150,NULL,0.8690,NULL,0.0000,NULL,26.5600,NULL,3,2013-01-02 03:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.0250,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2.3820,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-02 07:00:00,-0.3470,NULL,0.1750,NULL,64.3200,NULL,9.4500,NULL,28.3700,NULL,0.0000,NULL,0.6740,NULL,352.5000,NULL,34.3200,NULL,1.5150,NULL,0.7860,NULL,0.0000,NULL,25.6900,NULL,3,2013-01-02 07:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.0310,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2.9260,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-02 08:00:00,-0.4400,NULL,0.0530,NULL,64.2200,NULL,106.4000,NULL,26.0000,NULL,0.0000,NULL,0.8180,NULL,316.4000,NULL,34.0600,NULL,1.5150,NULL,0.9410,NULL,0.0000,NULL,25.3400,NULL,3,2013-01-02 08:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.7330,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2.8350,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-02 15:00:00,1.4860,NULL,-0.1930,NULL,44.6300,S,182.8000,NULL,33.6900,NULL,0.0000,NULL,1.1140,NULL,344.2000,NULL,51.0900,NULL,1.5150,NULL,1.6140,NULL,0.0000,NULL,26.6900,NULL,3,2013-01-02 15:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.7130,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1.3170,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-03 05:00:00,-4.1270,NULL,0.8360,NULL,72.7900,NULL,0.4310,NULL,16.1500,NULL,0.0000,NULL,0.1280,NULL,58.4300,NULL,26.9800,NULL,1.5150,NULL,0.1550,NULL,0.0000,NULL,21.0500,NULL,3,2013-01-03 05:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,-0.1450,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1.1050,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-03 07:00:00,-6.6340,NULL,0.9130,NULL,89.9000,NULL,8.1600,NULL,9.4100,NULL,0.0000,NULL,0.0560,NULL,189.2000,NULL,40.6700,NULL,1.5150,NULL,0.0950,NULL,0.0000,NULL,20.2300,NULL,3,2013-01-03 07:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.1990,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0.3450,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-03 13:00:00,1.9400,NULL,-0.5210,NULL,46.8700,S,347.6000,NULL,27.5900,NULL,0.0000,NULL,2.6770,NULL,257.6000,NULL,20.7400,NULL,1.5150,NULL,2.8360,NULL,0.0000,NULL,23.9700,NULL,3,2013-01-03 13:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,4.2330,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1.9130,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-03 15:00:00,3.0470,NULL,-0.2550,NULL,42.1200,S,215.0000,NULL,29.6400,NULL,0.0000,NULL,3.0220,NULL,250.3000,NULL,13.3000,NULL,1.5150,NULL,3.1190,NULL,0.0000,NULL,25.1300,NULL,3,2013-01-03 15:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,2.1830,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1.2930,NULL,39.028177,-76.817127,47.00000,Range,Flat
BEL116,2013-01-03 17:00:00,-0.0860,NULL,1.3010,NULL,59.2700,NULL,1.1100,NULL,15.1700,NULL,0.0000,NULL,0.1830,NULL,171.8000,NULL,48.8300,NULL,1.5150,NULL,0.4910,NULL,0.0000,NULL,25.0200,NULL,3,2013-01-03 17:00:00,metdata_2013.csv,NULL,NULL,NULL,NULL,NULL,0.0330,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0.995